In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu
import warnings
warnings.filterwarnings('ignore')

# Set style for publication-quality plots
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9

In [ ]:
# Load the study data
data_path = "../data/study_summary.xlsx"
df = pd.read_excel(data_path)

# Display basic info about the dataset
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()

In [ ]:
# Explore group distribution
print("Group distribution:")
print(df['Grupo'].value_counts())
print("\nGroup percentages:")
print(df['Grupo'].value_counts(normalize=True) * 100)

# Check for missing values in key demographic variables
demographic_vars = ['Edad', 'Genero', 'BMI', 'Grupo']
print("\nMissing values in demographic variables:")
for var in demographic_vars:
    if var in df.columns:
        missing_count = df[var].isnull().sum()
        missing_pct = (missing_count / len(df)) * 100
        print(f"{var}: {missing_count} ({missing_pct:.1f}%)")
    else:
        print(f"{var}: Column not found")

In [ ]:
# Function to compute demographic statistics
def compute_demographic_stats(df, group_col='Grupo'):
    """
    Compute demographic statistics for each group
    """
    results = {}
    
    # Age statistics
    if 'Edad' in df.columns:
        age_stats = df.groupby(group_col)['Edad'].agg(['count', 'mean', 'std', 'median']).round(2)
        results['age'] = age_stats
        print("Age Statistics:")
        print(age_stats)
        print()
    
    # Gender distribution
    if 'Genero' in df.columns:
        gender_crosstab = pd.crosstab(df[group_col], df['Genero'], margins=True)
        gender_pct = pd.crosstab(df[group_col], df['Genero'], normalize='index') * 100
        results['gender_counts'] = gender_crosstab
        results['gender_percentages'] = gender_pct.round(1)
        print("Gender Distribution (counts):")
        print(gender_crosstab)
        print("\nGender Distribution (percentages):")
        print(gender_pct.round(1))
        print()
    
    # BMI statistics
    if 'BMI' in df.columns:
        bmi_stats = df.groupby(group_col)['BMI'].agg(['count', 'mean', 'std', 'median']).round(2)
        results['bmi'] = bmi_stats
        print("BMI Statistics:")
        print(bmi_stats)
        print()
    
    return results

# Compute demographics
demographic_results = compute_demographic_stats(df)

In [ ]:
# Statistical tests for demographic differences
def perform_demographic_tests(df, group_col='Grupo'):
    """
    Perform statistical tests for group differences in demographics
    """
    results = {}
    
    # Filter for COVID vs CONTROL groups
    covid_data = df[df[group_col] == 'COVID']
    control_data = df[df[group_col] == 'CONTROL']
    
    print("=== STATISTICAL TESTS FOR GROUP DIFFERENCES ===\n")
    
    # Age comparison (t-test or Mann-Whitney U)
    if 'Edad' in df.columns:
        covid_age = covid_data['Edad'].dropna()
        control_age = control_data['Edad'].dropna()
        
        # Check normality (Shapiro-Wilk test)
        shapiro_covid = stats.shapiro(covid_age)
        shapiro_control = stats.shapiro(control_age)
        
        print(f"Age normality tests:")
        print(f"COVID: W={shapiro_covid.statistic:.4f}, p={shapiro_covid.pvalue:.4f}")
        print(f"CONTROL: W={shapiro_control.statistic:.4f}, p={shapiro_control.pvalue:.4f}")
        
        # Use t-test if both normal, otherwise Mann-Whitney U
        if shapiro_covid.pvalue > 0.05 and shapiro_control.pvalue > 0.05:
            stat, p_val = ttest_ind(covid_age, control_age)
            test_type = "Independent t-test"
        else:
            stat, p_val = mannwhitneyu(covid_age, control_age, alternative='two-sided')
            test_type = "Mann-Whitney U test"
        
        results['age_test'] = {'statistic': stat, 'p_value': p_val, 'test_type': test_type}
        print(f"\nAge comparison ({test_type}):")
        print(f"Statistic: {stat:.4f}, p-value: {p_val:.4f}")
        
    # Gender comparison (Chi-square test)
    if 'Genero' in df.columns:
        contingency_table = pd.crosstab(df[group_col], df['Genero'])
        chi2, p_val, dof, expected = chi2_contingency(contingency_table)
        
        results['gender_test'] = {
            'chi2': chi2, 
            'p_value': p_val, 
            'dof': dof,
            'contingency_table': contingency_table
        }
        print(f"\nGender distribution (Chi-square test):")
        print(f"Chi2: {chi2:.4f}, p-value: {p_val:.4f}, df: {dof}")
        
    # BMI comparison
    if 'BMI' in df.columns:
        covid_bmi = covid_data['BMI'].dropna()
        control_bmi = control_data['BMI'].dropna()
        
        if len(covid_bmi) > 0 and len(control_bmi) > 0:
            # Check normality
            shapiro_covid_bmi = stats.shapiro(covid_bmi)
            shapiro_control_bmi = stats.shapiro(control_bmi)
            
            if shapiro_covid_bmi.pvalue > 0.05 and shapiro_control_bmi.pvalue > 0.05:
                stat, p_val = ttest_ind(covid_bmi, control_bmi)
                test_type = "Independent t-test"
            else:
                stat, p_val = mannwhitneyu(covid_bmi, control_bmi, alternative='two-sided')
                test_type = "Mann-Whitney U test"
            
            results['bmi_test'] = {'statistic': stat, 'p_value': p_val, 'test_type': test_type}
            print(f"\nBMI comparison ({test_type}):")
            print(f"Statistic: {stat:.4f}, p-value: {p_val:.4f}")
    
    return results

# Perform statistical tests
test_results = perform_demographic_tests(df)

In [ ]:
# Identify quality of life and questionnaire columns
print("All columns in the dataset:")
print(df.columns.tolist())
print("\n" + "="*50)

# Look for quality of life related columns
qol_keywords = ['euroqol', 'eq5d', 'eq-5d', 'sleep', 'fas', 'fatigue', 'quality', 'vida', 
                'depression', 'anxiety', 'psych', 'mental', 'physical', 'social', 'scale']

qol_columns = []
for col in df.columns:
    col_lower = col.lower()
    if any(keyword in col_lower for keyword in qol_keywords):
        qol_columns.append(col)

print(f"Potential Quality of Life columns found ({len(qol_columns)}):")
for col in qol_columns:
    print(f"- {col}")

# Show basic statistics for QoL columns
if qol_columns:
    print(f"\nBasic statistics for QoL variables:")
    qol_data = df[qol_columns + ['Grupo']].copy()
    print(qol_data.describe())

In [ ]:
# Function to analyze quality of life metrics
def analyze_qol_metrics(df, qol_columns, group_col='Grupo'):
    """
    Analyze quality of life metrics by group
    """
    qol_results = {}
    
    print("=== QUALITY OF LIFE METRICS ANALYSIS ===\n")
    
    for col in qol_columns:
        print(f"\n--- {col} ---")
        
        # Basic statistics by group
        stats_by_group = df.groupby(group_col)[col].agg(['count', 'mean', 'std', 'median']).round(2)
        print("Statistics by group:")
        print(stats_by_group)
        
        # Statistical test
        covid_data = df[df[group_col] == 'COVID'][col].dropna()
        control_data = df[df[group_col] == 'CONTROL'][col].dropna()
        
        if len(covid_data) > 0 and len(control_data) > 0:
            # Check normality
            try:
                if len(covid_data) >= 3 and len(control_data) >= 3:
                    shapiro_covid = stats.shapiro(covid_data)
                    shapiro_control = stats.shapiro(control_data)
                    
                    # Choose appropriate test
                    if shapiro_covid.pvalue > 0.05 and shapiro_control.pvalue > 0.05:
                        stat, p_val = ttest_ind(covid_data, control_data)
                        test_type = "t-test"
                    else:
                        stat, p_val = mannwhitneyu(covid_data, control_data, alternative='two-sided')
                        test_type = "Mann-Whitney U"
                    
                    print(f"Statistical test ({test_type}): statistic={stat:.4f}, p-value={p_val:.4f}")
                    
                    qol_results[col] = {
                        'stats_by_group': stats_by_group,
                        'test_statistic': stat,
                        'p_value': p_val,
                        'test_type': test_type
                    }
                else:
                    print("Insufficient data for statistical testing")
            except Exception as e:
                print(f"Error in statistical testing: {e}")
        else:
            print("No data available for comparison")
        
        print("-" * 40)
    
    return qol_results

# Analyze QoL metrics if any were found
if qol_columns:
    qol_analysis = analyze_qol_metrics(df, qol_columns)
else:
    print("No quality of life columns identified. Let's examine the data more carefully...")

In [ ]:
# Create demographic plots for publication
def create_demographic_plots(df, save_path="../data/plots/", group_col='Grupo'):
    """
    Create publication-quality demographic plots
    """
    import os
    
    # Create plots directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Set up the plotting style
    plt.style.use('default')
    fig = plt.figure(figsize=(15, 10))
    
    # Define colors for groups
    colors = {'COVID': '#E74C3C', 'CONTROL': '#3498DB'}
    
    # Plot 1: Age distribution
    if 'Edad' in df.columns:
        plt.subplot(2, 3, 1)
        for group in df[group_col].unique():
            if pd.notna(group):
                data = df[df[group_col] == group]['Edad'].dropna()
                plt.hist(data, alpha=0.7, label=group, color=colors.get(group, 'gray'), 
                        bins=15, edgecolor='black', linewidth=0.5)
        plt.xlabel('Age (years)')
        plt.ylabel('Frequency')
        plt.title('Age Distribution by Group')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Box plot for age
        plt.subplot(2, 3, 2)
        age_data = [df[df[group_col] == group]['Edad'].dropna() for group in ['COVID', 'CONTROL']]
        box_plot = plt.boxplot(age_data, labels=['COVID', 'CONTROL'], patch_artist=True)
        for patch, color in zip(box_plot['boxes'], [colors['COVID'], colors['CONTROL']]):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        plt.ylabel('Age (years)')
        plt.title('Age Comparison')
        plt.grid(True, alpha=0.3)
    
    # Plot 2: Gender distribution
    if 'Genero' in df.columns:
        plt.subplot(2, 3, 3)
        gender_counts = pd.crosstab(df[group_col], df['Genero'])
        gender_pct = pd.crosstab(df[group_col], df['Genero'], normalize='index') * 100
        
        x = np.arange(len(gender_counts.index))
        width = 0.35
        
        if 'F' in gender_counts.columns and 'M' in gender_counts.columns:
            plt.bar(x - width/2, gender_pct['F'], width, label='Female', 
                   color='#E8B4BC', edgecolor='black', linewidth=0.5)
            plt.bar(x + width/2, gender_pct['M'], width, label='Male', 
                   color='#87CEEB', edgecolor='black', linewidth=0.5)
        
        plt.xlabel('Group')
        plt.ylabel('Percentage (%)')
        plt.title('Gender Distribution by Group')
        plt.xticks(x, gender_counts.index)
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # Plot 3: BMI distribution
    if 'BMI' in df.columns:
        plt.subplot(2, 3, 4)
        for group in df[group_col].unique():
            if pd.notna(group):
                data = df[df[group_col] == group]['BMI'].dropna()
                if len(data) > 0:
                    plt.hist(data, alpha=0.7, label=group, color=colors.get(group, 'gray'), 
                            bins=15, edgecolor='black', linewidth=0.5)
        plt.xlabel('BMI (kg/m²)')
        plt.ylabel('Frequency')
        plt.title('BMI Distribution by Group')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Box plot for BMI
        plt.subplot(2, 3, 5)
        bmi_data = []
        labels = []
        for group in ['COVID', 'CONTROL']:
            group_bmi = df[df[group_col] == group]['BMI'].dropna()
            if len(group_bmi) > 0:
                bmi_data.append(group_bmi)
                labels.append(group)
        
        if len(bmi_data) > 1:
            box_plot = plt.boxplot(bmi_data, labels=labels, patch_artist=True)
            for i, (patch, label) in enumerate(zip(box_plot['boxes'], labels)):
                patch.set_facecolor(colors.get(label, 'gray'))
                patch.set_alpha(0.7)
        plt.ylabel('BMI (kg/m²)')
        plt.title('BMI Comparison')
        plt.grid(True, alpha=0.3)
    
    # Summary statistics table as subplot
    plt.subplot(2, 3, 6)
    plt.axis('off')
    
    # Create summary table
    summary_text = "DEMOGRAPHIC SUMMARY\\n\\n"
    if 'Edad' in df.columns:
        covid_age = df[df[group_col] == 'COVID']['Edad'].describe()
        control_age = df[df[group_col] == 'CONTROL']['Edad'].describe()
        summary_text += f"Age (years):\\n"
        summary_text += f"COVID: {covid_age['mean']:.1f} ± {covid_age['std']:.1f}\\n"
        summary_text += f"CONTROL: {control_age['mean']:.1f} ± {control_age['std']:.1f}\\n\\n"
    
    if 'BMI' in df.columns:
        covid_bmi = df[df[group_col] == 'COVID']['BMI'].describe()
        control_bmi = df[df[group_col] == 'CONTROL']['BMI'].describe()
        summary_text += f"BMI (kg/m²):\\n"
        summary_text += f"COVID: {covid_bmi['mean']:.1f} ± {covid_bmi['std']:.1f}\\n"
        summary_text += f"CONTROL: {control_bmi['mean']:.1f} ± {control_bmi['std']:.1f}\\n\\n"
    
    plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "demographic_analysis.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Create demographic plots
demographic_fig = create_demographic_plots(df)

In [ ]:
# Create quality of life plots
def create_qol_plots(df, qol_columns, save_path="../data/plots/", group_col='Grupo'):
    """
    Create publication-quality plots for quality of life metrics
    """
    import os
    
    if not qol_columns:
        print("No quality of life columns to plot")
        return None
    
    # Create plots directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Calculate number of subplots needed
    n_vars = len(qol_columns)
    ncols = 3 if n_vars > 3 else n_vars
    nrows = int(np.ceil(n_vars / ncols))
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows))
    if n_vars == 1:
        axes = [axes]
    elif nrows == 1:
        axes = axes if n_vars > 1 else [axes]
    else:
        axes = axes.flatten()
    
    colors = {'COVID': '#E74C3C', 'CONTROL': '#3498DB'}
    
    for i, col in enumerate(qol_columns):
        ax = axes[i]
        
        # Create box plot
        data_for_plot = []
        labels_for_plot = []
        
        for group in ['COVID', 'CONTROL']:
            group_data = df[df[group_col] == group][col].dropna()
            if len(group_data) > 0:
                data_for_plot.append(group_data)
                labels_for_plot.append(group)
        
        if len(data_for_plot) > 0:
            box_plot = ax.boxplot(data_for_plot, labels=labels_for_plot, patch_artist=True)
            
            # Color the boxes
            for j, (patch, label) in enumerate(zip(box_plot['boxes'], labels_for_plot)):
                patch.set_facecolor(colors.get(label, 'gray'))
                patch.set_alpha(0.7)
                patch.set_edgecolor('black')
            
            # Add individual points
            for j, (data, label) in enumerate(zip(data_for_plot, labels_for_plot)):
                x_pos = j + 1
                ax.scatter(np.random.normal(x_pos, 0.04, len(data)), data, 
                          alpha=0.6, s=20, color='black', zorder=3)
        
        ax.set_title(col, fontweight='bold', fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.set_ylabel('Score')
        
        # Add statistical annotation if available
        if 'qol_analysis' in locals() and col in qol_analysis:
            p_val = qol_analysis[col]['p_value']
            if p_val < 0.001:
                sig_text = "p < 0.001***"
            elif p_val < 0.01:
                sig_text = f"p = {p_val:.3f}**"
            elif p_val < 0.05:
                sig_text = f"p = {p_val:.3f}*"
            else:
                sig_text = f"p = {p_val:.3f}ns"
            
            ax.text(0.02, 0.98, sig_text, transform=ax.transAxes, 
                   verticalalignment='top', fontsize=9,
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    # Hide unused subplots
    for i in range(len(qol_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "quality_of_life_analysis.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Create QoL plots if we have QoL columns
if qol_columns:
    qol_fig = create_qol_plots(df, qol_columns)
else:
    print("No QoL columns found for plotting")

In [ ]:
# Generate comprehensive summary table for publication
def create_summary_table(df, qol_columns=None, group_col='Grupo'):
    """
    Create a comprehensive summary table for publication
    """
    print("\\n" + "="*80)
    print("COMPREHENSIVE SUMMARY TABLE FOR PUBLICATION")
    print("="*80)
    
    # Demographics table
    demographics_summary = []
    
    # Sample sizes
    sample_sizes = df[group_col].value_counts()
    demographics_summary.append({
        'Variable': 'Sample size, n',
        'COVID': f"{sample_sizes.get('COVID', 0)}",
        'CONTROL': f"{sample_sizes.get('CONTROL', 0)}",
        'p-value': '-',
        'Test': '-'
    })
    
    # Age
    if 'Edad' in df.columns:
        covid_age = df[df[group_col] == 'COVID']['Edad'].dropna()
        control_age = df[df[group_col] == 'CONTROL']['Edad'].dropna()
        
        covid_age_str = f"{covid_age.mean():.1f} ± {covid_age.std():.1f}"
        control_age_str = f"{control_age.mean():.1f} ± {control_age.std():.1f}"
        
        # Get p-value from previous test
        p_val = test_results.get('age_test', {}).get('p_value', np.nan)
        test_type = test_results.get('age_test', {}).get('test_type', 'N/A')
        
        demographics_summary.append({
            'Variable': 'Age, years (mean ± SD)',
            'COVID': covid_age_str,
            'CONTROL': control_age_str,
            'p-value': f"{p_val:.3f}" if not np.isnan(p_val) else 'N/A',
            'Test': test_type
        })
    
    # Gender
    if 'Genero' in df.columns:
        gender_crosstab = pd.crosstab(df[group_col], df['Genero'])
        gender_pct = pd.crosstab(df[group_col], df['Genero'], normalize='index') * 100
        
        if 'F' in gender_crosstab.columns:
            covid_female = f"{gender_crosstab.loc['COVID', 'F']} ({gender_pct.loc['COVID', 'F']:.1f}%)"
            control_female = f"{gender_crosstab.loc['CONTROL', 'F']} ({gender_pct.loc['CONTROL', 'F']:.1f}%)"
        else:
            covid_female = "N/A"
            control_female = "N/A"
        
        p_val = test_results.get('gender_test', {}).get('p_value', np.nan)
        
        demographics_summary.append({
            'Variable': 'Female, n (%)',
            'COVID': covid_female,
            'CONTROL': control_female,
            'p-value': f"{p_val:.3f}" if not np.isnan(p_val) else 'N/A',
            'Test': 'Chi-square'
        })
    
    # BMI
    if 'BMI' in df.columns:
        covid_bmi = df[df[group_col] == 'COVID']['BMI'].dropna()
        control_bmi = df[df[group_col] == 'CONTROL']['BMI'].dropna()
        
        if len(covid_bmi) > 0 and len(control_bmi) > 0:
            covid_bmi_str = f"{covid_bmi.mean():.1f} ± {covid_bmi.std():.1f}"
            control_bmi_str = f"{control_bmi.mean():.1f} ± {control_bmi.std():.1f}"
            
            p_val = test_results.get('bmi_test', {}).get('p_value', np.nan)
            test_type = test_results.get('bmi_test', {}).get('test_type', 'N/A')
            
            demographics_summary.append({
                'Variable': 'BMI, kg/m² (mean ± SD)',
                'COVID': covid_bmi_str,
                'CONTROL': control_bmi_str,
                'p-value': f"{p_val:.3f}" if not np.isnan(p_val) else 'N/A',
                'Test': test_type
            })
    
    # Convert to DataFrame and display
    demographics_df = pd.DataFrame(demographics_summary)
    print("\\nDEMOGRAPHICS TABLE:")
    print(demographics_df.to_string(index=False))
    
    # Quality of life table
    if qol_columns and 'qol_analysis' in locals():
        print("\\n\\nQUALITY OF LIFE METRICS:")
        qol_summary = []
        
        for col in qol_columns:
            if col in qol_analysis:
                stats = qol_analysis[col]['stats_by_group']
                p_val = qol_analysis[col]['p_value']
                test_type = qol_analysis[col]['test_type']
                
                covid_stats = f"{stats.loc['COVID', 'mean']:.2f} ± {stats.loc['COVID', 'std']:.2f}"
                control_stats = f"{stats.loc['CONTROL', 'mean']:.2f} ± {stats.loc['CONTROL', 'std']:.2f}"
                
                qol_summary.append({
                    'Variable': col,
                    'COVID': covid_stats,
                    'CONTROL': control_stats,
                    'p-value': f"{p_val:.3f}",
                    'Test': test_type
                })
        
        if qol_summary:
            qol_df = pd.DataFrame(qol_summary)
            print(qol_df.to_string(index=False))
    
    # Save tables to CSV
    demographics_df.to_csv("../data/results/demographics_table.csv", index=False)
    if qol_columns and 'qol_analysis' in locals() and qol_summary:
        qol_df.to_csv("../data/results/qol_table.csv", index=False)
    
    return demographics_df

# Create summary table
summary_table = create_summary_table(df, qol_columns)

# Demographic and Quality of Life Analysis Script

This notebook provides a comprehensive analysis of demographic characteristics and quality of life metrics for the COVID neuroimaging cohort study.

## Features:

1. **Demographic Analysis**:
   - Age, gender, and BMI comparisons between COVID and CONTROL groups
   - Statistical tests (t-test/Mann-Whitney U for continuous, Chi-square for categorical)
   - Publication-quality visualizations

2. **Quality of Life Metrics**:
   - Automatic detection of QoL-related variables (EuroQol, sleep, fatigue, etc.)
   - Group comparisons with appropriate statistical tests
   - Box plots with individual data points

3. **Publication Outputs**:
   - High-resolution plots (300 DPI)
   - Summary tables in CSV format
   - Comprehensive statistical reporting

## Usage:
Run all cells in sequence. The script will automatically:
- Load data from `../data/study_summary.xlsx`
- Identify relevant variables
- Perform statistical analyses
- Generate plots and tables
- Save results to `../data/plots/` and `../data/results/`